Copyright (C) 2022, Intel Corporation

SPDX-License-Identifier: MIT License

# Image Classification with resnet in Python using torch-ort inference module:

1. This demo shows how to use Intel® OpenVINO™ integration with Torch-ORT to classify objects in images with ONNX Runtime OpenVINO Execution Provider.

2. We use an image classification model [ResNet-50](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.resnet50) from Torchvision and [ImageNet labels](https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt) to classify objects. In the labels file, you'll find 1,000 different categories that were used in the Imagenet competition.

### Import Necessary Resources

In [ ]:
import os
import time
import torch
import wget
import argparse
from PIL import Image
from torchvision import transforms
import torchvision.models as models
from torch_ort import ORTInferenceModule, OpenVINOProviderOptions

### download labels function
* Download the imagenet classes file from github

In [ ]:
def download_labels(labels):
    if not labels:
        labels = "imagenet_classes.txt"
        if not os.path.exists(labels):
            labelsUrl = (
                "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
            )
            # Download the file (if we haven't already)
            wget.download(labelsUrl)
        else:
            print("\nReusing downloaded imagenet labels")

    # Read the categories
    with open(labels, "r") as f:
        categories = [s.strip() for s in f.readlines()]
        return categories    

### Preprocess Function
* Resize the input
* crop the input
* Convert the image input to a tensor and normalize

In [ ]:
def preprocess(img):
    transform = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    return transform(img)

### Inference Function
* Runs the inference on the input image
* Prints the inference results

In [ ]:
def infer(model, image, categories):
    # warmup
    model(image)

    # Start inference
    t0 = time.time()
    outputs = model(image)
    t1 = time.time() - t0
    print("\nInference time: {:.4f}ms\n".format(t1 * 1000))

    # The output has unnormalized scores. Run a softmax on it for probabilities.
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

    # Show top categories per image
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    print("Top 5 Results: \nLabels , Probabilities:")
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())

### Get the input

In [ ]:
#Create List of files in the directory
files = os.listdir('.')

#Get the neccesary files into the directory if they don't already exist
if ('plane.jpg' not in files):
  !wget https://media.wired.com/photos/62b25f4c18e6fafaa97a6477/master/pass/Air-Serbia-Plane-Russian-Sanctions-Safety-Hazard-Business-1239498184.jpg -O plane.jpg

--2022-08-18 16:35:40--  https://media.wired.com/photos/62b25f4c18e6fafaa97a6477/master/pass/Air-Serbia-Plane-Russian-Sanctions-Safety-Hazard-Business-1239498184.jpg
Resolving media.wired.com (media.wired.com)... 151.101.0.239, 151.101.64.239, 151.101.128.239, ...
Connecting to media.wired.com (media.wired.com)|151.101.0.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 319129 (312K) [image/jpeg]
Saving to: ‘plane.jpg’

plane.jpg           100%[===================>] 311.65K  --.-KB/s    in 0.003s  

2022-08-18 16:35:40 (98.3 MB/s) - ‘plane.jpg’ saved [319129/319129]



### Select the inputs

* Use `input_file` to select your input image to be inferred

* Available backend precisions
  * CPU: FP32
  * GPU(Does not work in collab): FP32, FP16

* Available inference execution providers 
  * OpenVINO

In [ ]:
input_file = "plane.jpg"

backend ="CPU"
precision = "FP32"

### Setup inputs and labels for inference

In [ ]:
img = Image.open(input_file)
img_trans = preprocess(img)
# Adding batch dimension (size 1)
img_trans = torch.unsqueeze(img_trans, 0)

# Download ImageNet labels
labels = False
categories = download_labels(labels)

### Get and load the model

In [ ]:
# Download and load the model
model = models.resnet50(pretrained=True)
        
# Convert model for evaluation
model.eval();

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

### Run the inference with native PyTorch

In [ ]:
# Infer
infer(model, img_trans, categories)


Inference time: 234.3407ms

Top 5 Results: 
Labels , Probabilities:
airliner 0.9391021728515625
wing 0.0565822534263134
warplane 0.0038844654336571693
projectile 0.00012474275717977434
space shuttle 9.90280750556849e-05


### Run the inference with Torch-ORT inference module

In [ ]:
# Select OpenVINO as inference execution provider
if backend and precision:
    provider_options = OpenVINOProviderOptions(backend, precision)
    model_ort = ORTInferenceModule(model, provider_options=provider_options)
else:
    model_ort = ORTInferenceModule(model)

# Infer
infer(model_ort, img_trans, categories)
img.close()


Inference time: 133.7893ms

Top 5 Results: 
Labels , Probabilities:
airliner 0.9391021728515625
wing 0.0565822534263134
warplane 0.0038844728842377663
projectile 0.00012474264076445252
space shuttle 9.902826423058286e-05
